In [1]:
try:
    import google.colab
    IN_COLAB = True
    from tqdm.notebook import tqdm, trange

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/feature-circuits
    %pip install -r requirements.txt
    !git submodule update --init
except:
    IN_COLAB = False
    from tqdm import tqdm, trange

import os

import torch
from nnsight import LanguageModel

from circuit_without_tot import get_circuit, save_circuit
from circuit_plotting import plot_circuit
from dictionary_learning import AutoEncoder

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("DEVICE :", DEVICE)

print("IN_COLAB :", IN_COLAB)

DEVICE : cpu
IN_COLAB : False


In [2]:
# pythia70m = UnifiedTransformer("EleutherAI/pythia-70m-deduped", device=DEVICE)

# pythia70m_embed = pythia70m.embed

# pythia70m_resids= []
# pythia70m_attns = []
# pythia70m_mlps = []
# for layer in range(len(pythia70m.blocks)):
#     pythia70m_resids.append(pythia70m.blocks[layer])
#     pythia70m_attns.append(pythia70m.blocks[layer].attn)
#     pythia70m_mlps.append(pythia70m.blocks[layer].mlp)

In [3]:
pythia70m = LanguageModel("EleutherAI/pythia-70m-deduped", device_map=DEVICE)

with pythia70m.trace("I like choco"):
    out = pythia70m.gpt_neox.layers[-1].output.save()

print(out)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(tensor([[[3.7878, 1.5635, 2.2401,  ..., 3.4158, 1.7231, 3.2249],
         [5.5106, 3.8430, 4.1828,  ..., 3.9052, 2.9130, 4.8242],
         [3.2991, 1.3034, 1.5051,  ..., 2.6618, 2.3709, 1.4800],
         [4.0855, 2.0843, 2.0335,  ..., 2.8115, 2.1017, 2.7100]]],
       grad_fn=<AddBackward0>), (tensor([[[[ 2.6466e-01, -9.0403e-02, -1.2978e-01,  ..., -5.6741e+01,
            5.4417e+01, -5.9252e+01],
          [ 2.0098e+00,  2.9065e+00,  1.0229e+01,  ..., -5.6634e+01,
            5.4601e+01, -5.9340e+01],
          [-7.1375e+00,  6.7777e+00,  6.8730e+00,  ..., -5.7165e+01,
            5.3907e+01, -5.9449e+01],
          [-5.4164e+00,  4.7681e+00,  6.6199e+00,  ..., -5.6356e+01,
            5.4439e+01, -5.9809e+01]],

         [[-4.2213e-01, -1.8457e+00, -1.2065e+00,  ...,  5.4868e+01,
           -6.0050e+01, -5.7623e+01],
          [-5.2783e+00, -5.0359e+00, -9.8578e+00,  ...,  5.5199e+01,
           -6.0318e+01, -5.7770e+01],
          [ 7.1190e-01, -6.4730e+00, -9.2016e+00,  ...,  5.5

In [4]:
pythia70m = UnifiedTransformer("EleutherAI/pythia-70m-deduped", device=DEVICE, processing=False)

with pythia70m.trace("I like choco"):
    out_bis = pythia70m.blocks[-1].output.save()

print((out[0] - out_bis).abs().mean())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor(3.5246e-06, grad_fn=<MeanBackward0>)


In [5]:
pythia70m = LanguageModel("EleutherAI/pythia-70m-deduped", device_map=DEVICE, dispatch=True)

pythia70m_embed = pythia70m.gpt_neox.embed_in

pythia70m_resids= []
pythia70m_attns = []
pythia70m_mlps = []
for layer in range(len(pythia70m.gpt_neox.layers)):
    pythia70m_resids.append(pythia70m.gpt_neox.layers[layer])
    pythia70m_attns.append(pythia70m.gpt_neox.layers[layer].attention)
    pythia70m_mlps.append(pythia70m.gpt_neox.layers[layer].mlp)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# if IN_COLAB:
#     base = "/content/gdrive/MyDrive/feature-circuits/"
# else:
#     base = "C:/Users/Grégoire/Documents/ENS/stages/AttentionGraph/Marks/feature-circuits/"
# path = base + "dictionary_learning/dictionaires/pythia-70m-deduped/"

# if not os.path.exists(path):
#     if IN_COLAB:
#         # go to base / dictionary_learning :
#         %cd /content/gdrive/MyDrive/feature-circuits/dictionary_learning
#         !apt-get update
#         !apt-get install dos2unix
#         !dos2unix pretrained_dictionary_downloader.sh
#         !chmod +x pretrained_dictionary_downloader.sh
#         !./pretrained_dictionary_downloader.sh
#         %cd /content/gdrive/MyDrive/feature-circuits
#     else:
#         %cd C:/Users/Grégoire/Documents/ENS/stages/AttentionGraph/Marks/feature-circuits/dictionary_learning
#         %run ./pretrained_dictionary_downloader.sh
#         %cd C:/Users/Grégoire/Documents/ENS/stages/AttentionGraph/Marks/feature-circuits

# dictionaries = {}

# d_model = 512
# dict_size = 32768

# ae = AutoEncoder(d_model, dict_size)
# ae.load_state_dict(torch.load(path + f"embed/ae.pt", map_location='cpu'))
# dictionaries[pythia70m_embed] = ae.to(DEVICE)


# for layer in range(len(pythia70m.blocks)):
#     ae = AutoEncoder(d_model, dict_size)
#     ae.load_state_dict(torch.load(path + f"resid_out_layer{layer}/ae.pt", map_location='cpu'))
#     dictionaries[pythia70m_resids[layer]] = ae.to(DEVICE)

#     # ae = AutoEncoder(d_model, dict_size)
#     # ae.load_state_dict(torch.load(path + f"attn_out_layer{layer}/ae.pt", map_location='cpu'))
#     # dictionaries[pythia70m_attns[layer]] = ae.to(DEVICE)

#     # ae = AutoEncoder(d_model, dict_size)
#     # ae.load_state_dict(torch.load(path + f"mlp_out_layer{layer}/ae.pt", map_location='cpu'))
#     # dictionaries[pythia70m_mlps[layer]] = ae.to(DEVICE)

In [7]:
if IN_COLAB:
    base = "/content/gdrive/MyDrive/feature-circuits/"
else:
    base = "C:/Users/Grégoire/Documents/ENS/stages/AttentionGraph/Marks/feature-circuits/"
path = base + "dictionary_learning/dictionaires/pythia-70m-deduped/"

if not os.path.exists(path):
    if IN_COLAB:
        # go to base / dictionary_learning :
        %cd /content/gdrive/MyDrive/feature-circuits/dictionary_learning
        !apt-get update
        !apt-get install dos2unix
        !dos2unix pretrained_dictionary_downloader.sh
        !chmod +x pretrained_dictionary_downloader.sh
        !./pretrained_dictionary_downloader.sh
        %cd /content/gdrive/MyDrive/feature-circuits
    else:
        %cd C:/Users/Grégoire/Documents/ENS/stages/AttentionGraph/Marks/feature-circuits/dictionary_learning
        %run ./pretrained_dictionary_downloader.sh
        %cd C:/Users/Grégoire/Documents/ENS/stages/AttentionGraph/Marks/feature-circuits

dictionaries = {}

d_model = 512
dict_size = 32768

ae = AutoEncoder(d_model, dict_size).to(DEVICE)
ae.load_state_dict(torch.load(path + f"embed/ae.pt", map_location=DEVICE))
dictionaries[pythia70m_embed] = ae


for layer in range(len(pythia70m.gpt_neox.layers)):
    ae = AutoEncoder(d_model, dict_size).to(DEVICE)
    ae.load_state_dict(torch.load(path + f"resid_out_layer{layer}/ae.pt", map_location=DEVICE))
    dictionaries[pythia70m_resids[layer]] = ae

    # ae = AutoEncoder(d_model, dict_size).to(DEVICE)
    # ae.load_state_dict(torch.load(path + f"attn_out_layer{layer}/ae.pt", map_location=DEVICE))
    # dictionaries[pythia70m_attns[layer]] = ae

    # ae = AutoEncoder(d_model, dict_size).to(DEVICE)
    # ae.load_state_dict(torch.load(path + f"mlp_out_layer{layer}/ae.pt", map_location=DEVICE))
    # dictionaries[pythia70m_mlps[layer]] = ae

In [8]:
# def metric_fn_v1(model, trg=None):
#     """
#     default : return the logit
#     """
#     if trg is None:
#         raise ValueError("trg must be provided")
#     logits = model.unembed.output[:,-1,:]
#     return logits[torch.arange(trg.numel()), trg]

# def metric_fn_v2(model, trg=None):
#     """
#     Return -log probability for the expected target.

#     trg : torch.Tensor, contains idxs of the target tokens (between 0 and d_vocab_out)

#     /!\ here we assume that all last tokens are indeed in the last position (if padding, it must happen in front of the sequence, not after)
#     """
#     if trg is None:
#         raise ValueError("trg must be provided")
#     logits = model.unembed.output[:,-1,:]
#     return (
#          -1 * torch.gather(
#              torch.nn.functional.log_softmax(model.unembed.output[:,-1,:], dim=-1),
#              dim=-1, index=trg.view(-1, 1)
#          ).squeeze(-1)
#     )

In [9]:
def metric_fn_v1(model, trg=None):
    """
    default : return the logit
    """
    if trg is None:
        raise ValueError("trg must be provided")
    logits = model.embed_out.output[:,-1,:]
    return logits[torch.arange(trg.numel()), trg]
    
def metric_fn_v2(model, trg=None):
    """
    default : return the logit
    """
    if trg is None:
        raise ValueError("trg must be provided")
    logits = model.embed_out.output[:,trg[0],:]
    return logits[0, 0, trg[1]]

def metric_fn_v3(model, trg=None):
    """
    Return -log probability for the expected target.

    trg : torch.Tensor, contains idxs of the target tokens (between 0 and d_vocab_out)

    /!\ here we assume that all last tokens are indeed in the last position (if padding, it must happen in front of the sequence, not after)
    """
    if trg is None:
        raise ValueError("trg must be provided")
    logits = model.embed_out.output[:,-1,:]
    return (
         -1 * torch.gather(
             torch.nn.functional.log_softmax(model.embed_out.output[:,-1,:], dim=-1),
             dim=-1, index=trg.view(-1, 1)
         ).squeeze(-1)
    )

In [10]:
batch_size = 1

clean = [
    "When Mary and John went to the store, John gave a drink to"
    for _ in range(batch_size)
]
patch = None

trg = " Mary"
trg_idx = torch.tensor([pythia70m.tokenizer.encode(trg)[0]] * batch_size, device=DEVICE)
print(trg_idx)

tensor([6393])


In [11]:
circuit = get_circuit(
    clean, patch,
    pythia70m,
    pythia70m_embed, pythia70m_resids,
    dictionaries,
    metric_fn_v1, {"trg": trg_idx},
    edge_threshold=0.1
)

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Layer 5 : 2.504028081893921 seconds
Now processing layer 5 with 18 features
[428036, 428635, 430366, 432770, 433336, 435103, 437083, 444575, 445946, 449138, 449642, 451439, 452071, 454500, 457443, 458717, 458737, 458765]
Layer 4 : 71.19254660606384 seconds
Now processing layer 4 with 59 features
[426292, 427617, 428036, 428635, 429814, 430366, 430713, 431013, 431236, 431270, 432770, 432875, 433336, 433538, 433553, 434039, 435004, 435103, 435248, 437083, 437582, 437956, 438170, 438445, 440085, 440208, 441947, 442157, 442265, 442819, 442905, 443826, 444575, 444843, 444940, 445946, 446073, 446824, 447449, 447889, 449138, 449642, 449663, 449831, 450998, 451296, 451439, 451503, 452071, 452850, 453938, 454108, 454500, 455574, 455720, 457443, 458717, 458737, 458765]
Layer 3 : 192.45503115653992 seconds
Now processing layer 3 with 81 features
[426061, 426292, 427330, 427617, 427773, 428036, 428569, 428635, 429814, 430366, 430713, 431013, 431236, 431270, 432770, 432824, 432875, 433170, 433336, 

KeyboardInterrupt: 

- cpu :
    - 1 : 2m47
    - 2 : /
    - 10: Stop at 68m+

- gpu :
    - 1 : 42s
    - 2 : 1m32

In [ ]:
submod_1 = "resid_0"
submod_2 = "resid_1"

weights = circuit[1][submod_1][submod_2]
weights = weights.values()

from matplotlib import pyplot as plt

alive_downstream = circuit[1][submod_1][submod_2].indices()[0]
set_downstream = list(set([alive_downstream_.item() for alive_downstream_ in alive_downstream]))

ss = []
abss = []
nb_k = []

from tqdm import tqdm

for k in tqdm(set_downstream):
    weights = []
    for i, idx in enumerate(alive_downstream):
        if idx == k:
            weights.append(circuit[1][submod_1][submod_2].values()[i])
    weights = torch.stack(weights)

    perm = torch.argsort(weights.abs(), descending=True)
    weights = weights[perm]
    tot = sum(weights)
    s = 0
    for i in range(len(weights)):
        s += weights[i]
        if i < len(ss):
            ss[i] += (s / tot).item()
        else:
            ss.append((s / tot).item())
        if i < len(abss):
            abss[i] += weights[i].abs().item()
        else:
            abss.append(weights[i].abs().item())
        if i < len(nb_k):
            nb_k[i] += 1
        else:
            nb_k.append(1)
        # print("i :", i)
        # print("weight :", embed_weights[i].item())
        # print("% of total :", s.item() / tot.item() * 100)

ss = [ss[i] / nb_k[i] for i in range(len(ss))]
abss = [abss[i] / nb_k[i] for i in range(len(abss))]

"""
plot ss and abss on two different axis with the same x-axis on the same plot
"""
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('weight index')
ax1.set_ylabel('cumulative % of total', color=color)
ax1.plot(ss, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('weight', color=color)
ax2.plot(abss, color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.show()

In [ ]:
max_weights = 100

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('weight index')
ax1.set_ylabel('cumulative % of total', color=color)
ax1.plot(ss[:max_weights], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('weight', color=color)
ax2.plot(abss[:max_weights], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.show()

In [ ]:
import importlib
import circuit_plotting
importlib.reload(circuit_plotting)
circuit_plotting.plot_circuit(circuit[0], circuit[1], save_dir='./circuit/cpu_2_')

In [ ]:
all_weights = []
for key, value in circuit[1].items():
    for k, v in value.items():
        all_weights.append(v.values())
        
all_weights = torch.cat(all_weights, dim=0)
print(all_weights.shape)
print(all_weights.abs().mean())

plt.hist(all_weights[all_weights.abs() > 0.01].detach().cpu().numpy(), bins=100)
plt.show()

In [ ]:
A = torch.randn(1, 10, 50)
B = torch.randn(1, 10, 50)

print((A * B).shape)
print(A @ B)

In [ ]:
import torch

dummy_2d_sparse_idx = torch.tensor([[0, 99, 27], [1, 2, 199]])
dummy_2d_sparse_values = torch.randn(2, 3)

dummy_2d_sparse = torch.sparse_coo_tensor(
    dummy_2d_sparse_idx,
    dummy_2d_sparse_values,
    size=(100, 200)
)

print(dummy_2d_sparse.to_dense())